In [1]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.prompts import PromptTemplate
from rag import run_query, qa_chain, retriever, llm
from graphrag import answer_query_with_knowledge 
from langchain.agents import initialize_agent, Tool, AgentType

/home/tanisha.ramani@simform.dom/Desktop/github-posts/Traineework/Agents/rag.py:41: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectorstore = Chroma(


 Loaded vectorstore from disk.


In [2]:

# Define a tool for agent_A (Traditional RAG)
rag_tool = Tool(
    name="RAGTool",
    func=lambda query: run_query(qa_chain, retriever, query)["final_answer"],
    description="Use this for answering general medical queries based on text retrieval from NHS documents using vector search."
)

# Define a tool for agent_B (Graph-based RAG)
graphrag_tool = Tool(
    name="GraphBasedRAGTool",
    func=lambda query: answer_query_with_knowledge(query)["answer"],
    description="Use this for answering complex or multi-hop medical queries using a knowledge graph constructed from NHS documents."
)


In [3]:

# Initialize agent_A (Traditional RAG)
agent_A = initialize_agent(
    tools=[rag_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # or `openai-functions` for function-calling agents
    verbose=False
)

# Initialize agent_B (Graph-based RAG)
agent_B = initialize_agent(
    tools=[graphrag_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # or `openai-functions` for function-calling agents
    verbose=False
)

/tmp/ipykernel_120429/3191942374.py:2: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  agent_A = initialize_agent(


In [4]:


# Define a new tool for agent_C that compares answers from agent_A and agent_B
def compare_answers(query, answer_A, answer_B):
    # Example logic: Compare based on some predefined rules (e.g., length, clarity, etc.)
    if len(answer_A) > len(answer_B):  # Example criteria: longer answer
        return answer_A
    else:
        return answer_B

# Initialize agent_C (comparison agent)
compare_tool = Tool(
    name="ComparisonAgentTool",
    func=lambda query: compare_answers(query, *get_answers_from_agents(query)),
    description="Compares answers from both agents and returns the more satisfactory answer."
)


In [5]:

# Function to get answers from agent_A and agent_B for a given query
def get_answers_from_agents(query):
    # Get answer from agent_A (Traditional RAG)
    answer_A = agent_A.run(query)
    # Get answer from agent_B (Graph-based RAG)
    answer_B = agent_B.run(query)
    return answer_A, answer_B

# Now agent_C uses both agent_A and agent_B outputs
agent_C = initialize_agent(
    tools=[compare_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)


In [8]:
query = "What causes high blood sugar?"
final_answer = agent_C.run(query)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


High blood sugar, also known as hyperglycemia, can be caused by several factors. These include:

1. **Diabetes**: The most common cause of high blood sugar is diabetes, a condition where the body either doesn't produce enough insulin or can't effectively use the insulin it produces.

2. **Diet**: Consuming too many carbohydrates or sugary foods can lead to elevated blood sugar levels, especially if the body is unable to produce enough insulin to manage the sugar intake.

3. **Lack of Exercise**: Physical activity helps the body use insulin more efficiently. A sedentary lifestyle can contribute to higher blood sugar levels.

4. **Stress**: Physical or emotional stress can cause the body to release hormones that increase blood sugar levels.

5. **Illness**: Certain illnesses, particularly those that cause inflammation or infection, can lead to increased blood sugar levels.

6. **Medications**: Some medications, such as steroids, can raise blood sugar levels.

7. **Hormonal Changes**: Hor

In [9]:
print(f"Query: {query}")
    

print("\n--- Traditional RAG Response (Agent A) ---")
print(agent_A.run(query))

print("\n--- Graph-Based RAG Response (Agent B) ---")
print(agent_B.run(query))

print("\n--- Final Answer (from Comparison Agent) ---")
print(final_answer)
print("\n" + "-" * 80)

Query: What causes high blood sugar?

--- Traditional RAG Response (Agent A) ---
High blood sugar can be caused by insulin issues, lifestyle factors, family history, and diet or medication.

--- Graph-Based RAG Response (Agent B) ---
High blood sugar can be caused by diabetes, excessive carbohydrate or sugar intake, lack of physical activity, stress, illness, certain medications, and hormonal changes.

--- Final Answer (from Comparison Agent) ---
High blood sugar can be caused by diabetes, diet high in carbohydrates or sugar, lack of exercise, stress, illness, certain medications, hormonal changes, and pancreatic disorders.

--------------------------------------------------------------------------------
